### Prompts

Prompt allow you to create System Messages with input variables, for example, this:
**SystemMessage(content="You are a helpful assistant that translates the English to Spanish.")**
English and Spanish may be dynamic. This can be archieved with templates

In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [2]:
TEMPLATE = """
You are a helpful assistant that translates the {input_language} to {output_language}
"""

In [3]:
from langchain.prompts.prompt import PromptTemplate # Calling the PromptTemplate class from the langchain.prompts.prompt module

# Creating a PromptTemplate object from the TEMPLATE string
prompt_template = PromptTemplate.from_template( 
    template=TEMPLATE
)
prompt_template.format(input_language="english", output_language="german")

'\nYou are a helpful assistant that translates the english to german\n'

Passing the input_variables to the constructor will provide additional validation for the template

In [4]:
prompt_template = PromptTemplate(template=TEMPLATE, input_variables=["input_language", "output_language"])
prompt_template.format(input_language="english", output_language="german")


'\nYou are a helpful assistant that translates the english to german\n'

### Few Shot Prompt - provide a few examples in the template

In [5]:
TEMPLATE = """
Analyze the sentiment and subject of the given text.

Instructions:
1. Sentiment Analysis: Determine if the text expresses a positive, neutral, or negative sentiment
2. Subject Identification: Extract the main subject as a single word

Please provide the analysis in JSON format with these fields:
- sentiment: "positive", "neutral", or "negative"
- subject: single word describing the main topic

text: {input}
"""

To improve performance we can provide examples to increase the quality of the output.

In [6]:
TEMPLATE = """
Interpret the text and evaluate the text.
sentiment: is the text in a positive, neutral or negative sentiment?
subject: What subject is the text about? Use exactly one word.

Format the output as JSON with the following keys:
sentiment
subject

text: {input}

Examples:
text: The BellaVista restaurant offers an exquisite dining experience. The flavors are rich and the presentation is impeccable.
sentiment: positive
subject: BellaVista

text: BellaVista restaurant was alright. The food was decent, but nothing stood out.
sentiment: neutral
subject: BellaVista

text: I was disappointed with BellaVista. The service was slow and the dishes lacked flavor.
sentiment: negative
subject: BellaVista

text: SeoulSavor offered the most authentic Korean flavors I've tasted outside of Seoul. The kimchi was perfectly fermented and spicy.
sentiment: positive
subject: SeoulSavor

text: SeoulSavor was okay. The bibimbap was good but the bulgogi was a bit too sweet for my taste.
sentiment: neutral
subject: SeoulSavor

text: I didn't enjoy my meal at SeoulSavor. The tteokbokki was too mushy and the service was not attentive.
sentiment: negative
subject: SeoulSavor

text: MunichMeals has the best bratwurst and sauerkraut I've tasted outside of Bavaria. Their beer garden ambiance is truly authentic.
sentiment: positive
subject: MunichMeals

text: MunichMeals was alright. The weisswurst was okay, but I've had better elsewhere.
sentiment: neutral
subject: MunichMeals

text: I was let down by MunichMeals. The potato salad lacked flavor and the staff seemed uninterested.
sentiment: negative
subject: MunichMeals
"""


In [7]:
prompt_template = PromptTemplate(template=TEMPLATE, input_variables=["input"])
prompt_template.format(input="The MunichDeals experience was just awesome!")

"\nInterpret the text and evaluate the text.\nsentiment: is the text in a positive, neutral or negative sentiment?\nsubject: What subject is the text about? Use exactly one word.\n\nFormat the output as JSON with the following keys:\nsentiment\nsubject\n\ntext: The MunichDeals experience was just awesome!\n\nExamples:\ntext: The BellaVista restaurant offers an exquisite dining experience. The flavors are rich and the presentation is impeccable.\nsentiment: positive\nsubject: BellaVista\n\ntext: BellaVista restaurant was alright. The food was decent, but nothing stood out.\nsentiment: neutral\nsubject: BellaVista\n\ntext: I was disappointed with BellaVista. The service was slow and the dishes lacked flavor.\nsentiment: negative\nsubject: BellaVista\n\ntext: SeoulSavor offered the most authentic Korean flavors I've tasted outside of Seoul. The kimchi was perfectly fermented and spicy.\nsentiment: positive\nsubject: SeoulSavor\n\ntext: SeoulSavor was okay. The bibimbap was good but the bu

LangChain also provides a FewShotPromptTemplate class, which allows creating the examples more modularized

In [8]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

examples = [
    {
        "text": "The BellaVista restaurant offers an exquisite dining experience. The flavors are rich and the presentation is impeccable.",
        "response": "sentiment: positive\nsubject: BellaVista"
    },
    {
        "text": "BellaVista restaurant was alright. The food was decent, but nothing stood out.",
        "response": "sentiment: neutral\nsubject: BellaVista"
    },
    {
        "text": "I was disappointed with BellaVista. The pasta was overcooked and the sauces lacked flavor.",
        "response": "sentiment: negative\nsubject: BellaVista"
    },
    {
        "text": "SeoulSavor offered the most authentic Korean flavors I've tasted outside of Seoul. The kimchi was perfectly fermented and spicy.",
        "response": "sentiment: positive\nsubject: SeoulSavor"
    },
    {
        "text": "I didn't enjoy my meal at SeoulSavor. The tteokbokki was too mushy and the service was not attentive.",
        "response": "sentiment: negative\nsubject: SeoulSavor"
    },
    {
        "text": "MunichMeals has the best bratwurst and sauerkraut I've tasted outside of Bavaria. Their beer garden ambiance is truly authentic.",
        "response": "sentiment: positive\nsubject: MunichMeals"
    },
    {
        "text": "I was let down by MunichMeals. The potato salad lacked flavor and the staff seemed uninterested.",
        "response": "sentiment: negative\nsubject: MunichMeals"
    }
]

In [9]:
new_example = {
    "text": "SeoulSavor was okay. The bibimbap was good but the bulgogi was a bit too sweet for my taste.",
    "response": "sentiment: neutral\nsubject: SeoulSavor"
}
examples.append(new_example)

In [10]:
example_prompt = PromptTemplate(input_variables=["text", "response"], template="Text: {text}\n{response}")

In [11]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="text: {input}",
    input_variables=["input"]
)

In [12]:
print(prompt.format(input="The MunichDeals experience was just awesome!"))

Text: The BellaVista restaurant offers an exquisite dining experience. The flavors are rich and the presentation is impeccable.
sentiment: positive
subject: BellaVista

Text: BellaVista restaurant was alright. The food was decent, but nothing stood out.
sentiment: neutral
subject: BellaVista

Text: I was disappointed with BellaVista. The pasta was overcooked and the sauces lacked flavor.
sentiment: negative
subject: BellaVista

Text: SeoulSavor offered the most authentic Korean flavors I've tasted outside of Seoul. The kimchi was perfectly fermented and spicy.
sentiment: positive
subject: SeoulSavor

Text: I didn't enjoy my meal at SeoulSavor. The tteokbokki was too mushy and the service was not attentive.
sentiment: negative
subject: SeoulSavor

Text: MunichMeals has the best bratwurst and sauerkraut I've tasted outside of Bavaria. Their beer garden ambiance is truly authentic.
sentiment: positive
subject: MunichMeals

Text: I was let down by MunichMeals. The potato salad lacked flavo

### Chain-of-thought Prompting

Instead of just providing examples, we can also provide examples which include the whole thought process of why a review is negative, neutral or positive

In [13]:
TEMPLATE = """
Interpret the text and evaluate the text. Determine if the text has a positive, neutral, or negative sentiment. Also, identify the subject of the text in one word.

Format the output as JSON with the following keys:
sentiment
subject

text: {input}

Chain-of-Thought Prompts:
Let's start by evaluating a statement. Consider: "The BellaVista restaurant offers an exquisite dining experience. The flavors are rich and the presentation is impeccable." How does this make you feel about BellaVista?
 It sounds like a positive review for BellaVista.

Based on the positive nature of that statement, how would you format your response?
 { "sentiment": "positive", "subject": "BellaVista" }

Now, think about this: "SeoulSavor was okay. The bibimbap was good but the bulgogi was a bit too sweet for my taste." Does this give a strong feeling either way?
 Not particularly. It seems like a mix of good and not-so-good elements, so it's neutral.

Given the neutral sentiment, how should this be presented?
 { "sentiment": "neutral", "subject": "SeoulSavor" }

Lastly, ponder on this: "I was let down by MunichMeals. The potato salad lacked flavor and the staff seemed uninterested." What's the overall impression here?
 The statement is expressing disappointment and dissatisfaction.

And if you were to categorize this impression, what would it be?
 { "sentiment": "negative", "subject": "MunichMeals" }
"""

### Composition

You can also compose multiple prompts together. This can be especially useful if you want to reuse parts of prompts.

In [19]:
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.prompts.prompt import PromptTemplate

# Introduction
introduction_template = """
Interpret the text and evaluate the text. Determine if the text has a positive, neutral, or negative sentiment. Also, identify the subject of the text in one word.
"""
introduction_prompt = PromptTemplate.from_template(introduction_template)

# Example
example_template = """
Chain-of-Thought Prompts:
Let's start by evaluating a statement. Consider: "{example_text}". How does this make you feel about {example_subject}?
Response: {example_evaluation}

Based on the {example_sentiment} nature of that statement, how would you format your response?
Response: {example_format}
"""
example_prompt = PromptTemplate.from_template(example_template)

# Execution
execution_template = """
Now, execute this process for the text: "{input}".
"""
execution_prompt = PromptTemplate.from_template(execution_template)

# Composing the full prompt
full_template = """
{introduction}
{example}
{execution} """

full_prompt = PromptTemplate.from_template(full_template)

# PipelinePrompts
input_prompts = [
    ("introduction", introduction_prompt),
    ("example", example_prompt),
    ("execution", execution_prompt)
]
pipeline_prompt = PipelinePromptTemplate(final_prompt=full_prompt, pipeline_prompts=input_prompts)


In [20]:
print(pipeline_prompt.format(
    example_text="The BellaVista restaurant offers an exquisite dining experience. The flavors are rich and the presentation is impeccable.",
    example_subject="BellaVista",
    example_evaluation="It sounds like a positive review for BellaVista.",
    example_sentiment="positive",
    example_format='{ "sentiment": "positive", "subject": "BellaVista" }',
    input="The new restaurant downtown has bland dishes and the wait time is too long."
))



Interpret the text and evaluate the text. Determine if the text has a positive, neutral, or negative sentiment. Also, identify the subject of the text in one word.


Chain-of-Thought Prompts:
Let's start by evaluating a statement. Consider: "The BellaVista restaurant offers an exquisite dining experience. The flavors are rich and the presentation is impeccable.". How does this make you feel about BellaVista?
Response: It sounds like a positive review for BellaVista.

Based on the positive nature of that statement, how would you format your response?
Response: { "sentiment": "positive", "subject": "BellaVista" }


Now, execute this process for the text: "The new restaurant downtown has bland dishes and the wait time is too long.".
 


### Serializing prompts

In [16]:
prompt = PromptTemplate(input_variables=["input"], template="Tell me a joke about {input}")
prompt.save("prompt.yaml")
prompt.save("prompt.json")

In [17]:
from langchain.prompts import load_prompt

prompt = load_prompt("prompt.yaml")
prompt.format(input="chickens")

'Tell me a joke about chickens'

In [18]:
prompt = load_prompt("prompt.json")
prompt.format(input="cows")

'Tell me a joke about cows'